In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV

# Data Preprocessing and Model Training

This notebook processes Ethiopia LSMS (Living Standards Measurement Survey) 2015 data to create a consumption prediction model for poverty targeting.

**Pipeline:**
1. Load and clean survey data (consumption, assets, housing, ...)
2. Merge into a single household-level dataset
3. Train a LASSO model to predict consumption from observable characteristics

**Data source:** The survey data can be downloaded from the LSMS program [here](https://microdata.worldbank.org/index.php/catalog/2783).

**Output:** `survey2015.csv` — processed dataset with predictions

In [ ]:
root = 'PATH-TO_ROOT-FOLDER'

## Preprocessing 

### Consumption data

In [ ]:
# Read consumption data, get HHID + consumption HH weight columns
cons = pd.read_stata(root + 'cons_agg_w3.dta')
cons = cons[['household_id', 'nom_totcons_aeq', 'adulteq']]
cons.columns = ['hhid', 'Per Capita Consumption', 'Adults']
            
# Construct total consumption from per capita consumption
cons['Consumption'] = cons['Per Capita Consumption']*cons['Adults']
cons = cons[['hhid', 'Consumption']]

# Drop any households with missing ID or missing consumption
cons['hhid'] = cons['hhid'].apply(lambda x: np.nan if x == '' else x) 
cons = cons.dropna(how='any')

# Convert to Birr per year to USD PPP per year
exchange_rate = 20.13
cons['Consumption'] = cons['Consumption']/exchange_rate
print('Households with consumption data: %i' % len(cons))

Households with consumption data: 4717


### Asset data

In [4]:
# Get asset data from asset module 
assets = pd.read_stata(root + 'Household/sect10_hh_w3.dta')
assets = assets[['household_id', 'hh_s10q00', 'hh_s10q01']]
assets['household_id'] = assets['household_id'].apply(lambda x: np.nan if x == '' else x)

# Drop households missing ID and duplicate records
assets = assets.dropna(subset=['household_id'])
assets = assets.drop_duplicates(subset=['household_id', 'hh_s10q00'])

assets[assets['household_id'].isin(['13010100303004', '01050100105031', '13010100303032'])]

# Pivot to per-household dataset 
assets = assets.pivot(columns=['hh_s10q00'], index=['household_id'])
assets.columns = [str(c[1]) for c in assets.columns]
assets['hhid'] = assets.index
assets.index = range(len(assets))
assets = assets.fillna(0)
assets.columns = ['Kerosene stove', 'Gas stove', 'Electric stove', 'Blanket', 'Bed', 'Clock', 'Telephone',
                 'Mobile phone', 'Radio', 'TV', 'CD/DVD', 'Satellite dish', 'Sofa', 'Bicycle', 'Motorcycle',
                 'Hand cart', 'Animal-drawn cart', 'Sewing machine', 'Weaving equipment', 'Mitad', 
                  'Energy-saving stove', 'Refridgerator', 'Car', 'Jewels', 'Wardrobe', 'Shelf', 'Biogas stove',
                 'Water storage pit', 'Mofer and kember', 'Sickle', 'Axe', 'Pick axe', 'Plough (traditional)',
                 'Plough (modern)', 'Water pump', 'hhid']
print('Households with asset data: %i' % len(assets))

Households with asset data: 4951


### Housing data

In [5]:
# Get housing data from housing module
housing = pd.read_stata(root + 'Household/sect9_hh_w3.dta', convert_categoricals=False)
housing = housing[['household_id', 'hh_s9q03', 'hh_s9q04', 'hh_s9q05', 'hh_s9q06', 'hh_s9q07', 'hh_s9q08', 
                   'hh_s9q09',  'hh_s9q12', 'hh_s9q17', 'hh_s9q19_a', 'hh_s9q21']]
colnames = ['hhid', 'Ownership status', 'Rooms', 'Walls', 'Roof', 'Floor', 'Kitchen', 'Oven', 'Garbage', 
            'Other houses', 'Lighting fuel', 'Cooking fuel']
housing.columns = colnames

# Drop households missing IDs, no other column has missingness
housing['hhid'] = housing['hhid'].apply(lambda x: np.nan if x == '' else x)
housing = housing.dropna()

print('Households with housing data: %i' % len(housing))

Households with housing data: 4954


### Demographic data

In [6]:
# Get demographic data from individual rsoter module 
demographics = pd.read_stata(root + 'Household/sect1_hh_w3.dta', convert_categoricals=False)
demographics = demographics[['household_id', 'hh_s1q02', 'hh_s1q03', 'hh_s1q04a', 'hh_s1q04b', 'hh_s1q08']]
demographics.columns = ['hhid', 'hhh', 'gender', 'age1', 'age2', 'married']

# Get age -- use age of household head if available, infer household head age as second member's age if not
demographics['age'] = demographics.apply(lambda row: row['age1'] if not pd.isnull(row['age1'])
                                        else row['age2'] if not pd.isnull(row['age2'])
                                        else np.nan, axis=1)

# Add missing category to martical status
demographics['married'] = demographics['married'].fillna(0)

# Drop any rows missing ID
demographics['hhid'] = demographics['hhid'].apply(lambda x: np.nan if x == '' else x)
demographics = demographics.dropna(subset=['hhid'])

# Create dataframe of household head data
hhh = demographics[demographics['hhh'] == 1].copy()
hhh = hhh[['hhid', 'gender', 'age', 'married']]
hhh.columns = ['hhid', 'HHH gender', 'HHH age', 'HHH married']

# Create dataframe with number of children in household
children = demographics[demographics['age'] < 5].copy()
children = children[['hhid', 'gender']].groupby('hhid', as_index=False).agg('count')\
    .rename({'gender':'Children'}, axis=1)

# Create dataframe with other misc demographic data -- region, number of members
geo = pd.read_stata(root + 'cons_agg_w3.dta', convert_categoricals=False)
geo = geo[['household_id', 'saq01', 'hh_size']]
geo.columns = ['hhid', 'region', 'hh_size']

# Merge all demographic data together
demographics = geo.merge(hhh, on='hhid', how='left').merge(children, on='hhid', how='left')
demographics['Children'] = demographics['Children'].fillna(0)
demographics = demographics.dropna()
print('Households with demographic data: %i' % len(demographics))

Households with demographic data: 4953


### Education data

In [7]:
# Get IDs for just household heads
hhh = pd.read_stata(root + '/Household/sect1_hh_w3.dta', convert_categoricals=False)
hhh = hhh[['household_id', 'individual_id2', 'hh_s1q02']]
hhh.columns = ['hhid', 'iid', 'hhh']
hhh = hhh[hhh['hhh'] == 1][['hhid', 'iid']]

# Read education data from individual roster module 
education = pd.read_stata(root + '/Household/sect2_hh_w3.dta', convert_categoricals=False)
education = education[['individual_id2', 'hh_s2q02', 'hh_s2q03', 'hh_s2q05']]
education.columns = ['iid', 'literate', 'school', 'level_school']

# Merge education data to IDs to get education data for just household heads
education = education.merge(hhh, on='iid', how='inner').drop('iid', axis=1)

# Recode literacy and education variables 
education['literate'] = 1 - (education['literate'] - 1)
education['level_school'] = education.apply(lambda row: -1 if row['school'] == 2 else 
                                            row['level_school'], axis=1)
education = education.drop('school', axis=1)
education['level_school'] = education['level_school'].fillna(-1)

print('Households with education data: %i' % len(demographics))

Households with education data: 4953


### Merge all data together

In [8]:
# Combine together all dataframes (HH need to have consumption data, can have other data missing)
df = cons.merge(demographics, on='hhid', how='left').merge(housing, on='hhid', how='left')\
    .merge(assets, on='hhid', how='left').merge(education, on='hhid', how='left')\
    .dropna()\
    .drop_duplicates(subset=['hhid'])

# Get outcome and feature variables
outcome = 'Consumption'
x_vars = [c for c in df.columns if c not in ['hhid', outcome]]

# Define type for each feature variable manually 
x_var_types = ['cat', 'int', 'bin', 'int', 'cat', 'int', 'cat', 'int', 'cat', 'cat', 'cat', 'cat',
              'cat', 'cat', 'bin', 'cat', 'cat'] + ['int']*(len(assets.columns) - 1) + ['bin', 'cat']

for v, var in enumerate(x_vars):
    
    # Check binary variables for having exactly 2 values, and convert values to 0 and 1
    if x_var_types[v] == 'bin':
        if len(df[var].unique()) != 2:
            print('Error: Unique values in column ' + var + ' not equal to 2')
        if 2 in df[var].unique():
            df[var] = df[var] - 1
            
    # Winsorize then scale continuous variables, and drop any that have 100% missingness or are constant
    elif x_var_types[v] == 'int':
        cutoff = np.percentile(df[var], 99)
        df[var] = df[var].apply(lambda x: cutoff if x > cutoff else x)
        df[var] = (df[var] - df[var].min())/(df[var].max() - df[var].min())
        if df[var].isnull().mean() == 1:
            df = df.drop(var, axis=1)

# Combine any categories in categorical variables with <1% of observations into an "other" category
# and one hot encode
cat_vars = [x_vars[v] for v in range(len(x_vars)) if x_var_types[v] == 'cat']
for cat_var in cat_vars:
    to_remove = []
    grouped = pd.DataFrame(df.groupby(cat_var).count()['hhid']/len(df))
    for value, share in grouped.iterrows():
        if share[0] < 0.01:
            to_remove.append(value)
    df[cat_var] = df[cat_var].apply(lambda x: 9999 if x in to_remove else x)
df = pd.get_dummies(df, columns=cat_vars, drop_first=True)

df.to_csv('survey2015.csv', index=False)
print('Households in final dataset: %i' % len(df))

Households in final dataset: 4694


## Model Training

In [8]:
def train_consumption_model(random_state=0):
    """Train LASSO model to predict consumption from survey features.
    
    Returns:
        DataFrame with columns 'y_test' (actual) and 'yhat_test' (predicted),
        both in USD PPP.
    """
    df = pd.read_csv('survey2015.csv')
    
    outcome = 'Consumption'
    extras = ['hhid']
    
    # Train/test split, log-transform consumption
    train, test = train_test_split(df, random_state=random_state, test_size=0.25)
    x_train = train.drop([outcome] + extras, axis=1)
    y_train = np.log(train[outcome])
    x_test = test.drop([outcome] + extras, axis=1)
    y_test = np.log(test[outcome])
    
    # Train LASSO with cross-validated alpha
    model = LassoCV(fit_intercept=True, random_state=random_state)
    model.fit(x_train, y_train)
    yhat_test = model.predict(x_test)
    
    # Back-transform to USD PPP
    results = pd.DataFrame({
        'y_test': np.exp(y_test),
        'yhat_test': np.exp(yhat_test)
    })
    
    return results
